In [1]:
#%pip install -U spaCy
#%pip install keras
#%python -m spacy download en_core_web_sm
#%pip install -U jax jaxlib
#%pip install clean-text
#%pip install --upgrade ipykernel
#da eseguire al primo utilizzo

In [27]:
import string
import spacy
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from pandas import read_csv
import nltk
import os
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from sklearn.svm import LinearSVC
from sklearn.utils import compute_class_weight




  
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

#init spaCy
punctuations = string.punctuation

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    translator = str.maketrans("", "", string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation.lower()
    #return text.strip().lower()

def treebankWordTokenizer(sentence):

    tokenizer = TreebankWordTokenizer()
    lemmatizer = WordNetLemmatizer()

    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)

    # Lemmatize each token
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return lemmatized_tokens



Number of CPUs in the system: 8


In [28]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\perna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\perna\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [29]:
vector = CountVectorizer(tokenizer = treebankWordTokenizer, ngram_range=(1,2))

In [48]:
def evaluation_metrics(x, y, pipe):

    y_pred = pipe.predict(x)

    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred, average="macro")
    recall = recall_score(y, y_pred, average="macro")
    f1 = f1_score(y, y_pred, average="macro")

    print("Accuracy: {:.4f}".format(accuracy))
    print("Precision: {:.4f}".format(precision))
    print("Recall: {:.4f}".format(recall))
    print("F1 Score: {:.4f}".format(f1))

# Task A

In [31]:
dft = read_csv('../Dataset/Processed datasets/train_sexist.csv')
dfv = read_csv('../Dataset/Processed datasets/dev_sexist.csv')
dfs = read_csv('../Dataset/Processed datasets/test_sexist.csv')

x_train = dft['text']
y_train = dft['label_sexist']
x_val = dfv['text']
y_val = dfv['label_sexist']
x_test = dfs['text']
y_test= dfs['label_sexist']
dft.set_index('ID')
dfv.set_index('ID')
dfs.set_index('ID')
print("TRAIN: \n", y_train.value_counts(), end="\n\n")
print("VALIDATION: \n", y_val.value_counts(), end="\n\n")
print("TEST: \n", y_test.value_counts(), end="\n\n")

TRAIN: 
 not sexist    10602
sexist         3398
Name: label_sexist, dtype: int64

VALIDATION: 
 not sexist    1514
sexist         486
Name: label_sexist, dtype: int64

TEST: 
 not sexist    3030
sexist         970
Name: label_sexist, dtype: int64



In [64]:
classifier = svm.LinearSVC(max_iter = 10000, class_weight= {"not sexist": 0.2, "sexist": 1})

# Create the pipeline

pipe_sexism = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])

pipe_sexism.fit(x_train, y_train)


d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001BCAB9D40A0>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function treebankWordTokenizer at 0x000001BCE883B160>)),
                ('classifier',
                 LinearSVC(class_weight={'not sexist': 0.2, 'sexist': 1},
                           max_iter=10000))])

In [65]:
#evaluation metrics for the validation data

evaluation_metrics(x_val, y_val, pipe_sexism)

Accuracy: 0.8340
Precision: 0.7899
Recall: 0.7255
F1 Score: 0.7481


In [57]:
params = {'vectorizer__ngram_range': [(1,1),(1,2)],
          'classifier__C': [0.2, 0.4, 1],
          'classifier__class_weight': ['balanced', None]}

pipe_optimized = GridSearchCV(pipe_sexism,param_grid=params,cv=10,scoring=make_scorer(f1_score, average="macro"),n_jobs=n_cpu-1,refit=True)
pipe_optimized.fit(x_train, y_train)
print("Migliori iperparametri:",pipe_optimized.best_params_)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Migliori iperparametri: {'classifier__C': 0.2, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2)}


In [58]:
best_params = pipe_optimized.best_params_
pipe_sexism.set_params(**best_params)
pipe_sexism.fit(x_train, y_train)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001BCAE1E39D0>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<function treebankWordTokenizer at 0x000001BCE883B160>)),
                ('classifier',
                 LinearSVC(C=0.2, class_weight='balanced', max_iter=10000))])

In [59]:
#evaluation metrics for the test data

evaluation_metrics(x_test, y_test, pipe_sexism)

Accuracy: 0.8405
Precision: 0.7947
Recall: 0.7444
F1 Score: 0.7637


# Task B

In [12]:
dft = read_csv('../Dataset/Processed datasets/train_category.csv')
dfv = read_csv('../Dataset/Processed datasets/dev_category.csv')
dfs = read_csv('../Dataset/Processed datasets/test_category.csv')

x1_train = dft['text']
y1_train = dft['label_category']
x1_val = dfv['text']
y1_val = dfv['label_category']
x1_test = dfs['text']
y1_test= dfs['label_category']
dft.set_index('ID')
dfv.set_index('ID')
dfs.set_index('ID')

print("TRAIN: \n", y1_train.value_counts(), end="\n\n")
print("VALIDATION: \n", y1_val.value_counts(), end="\n\n")
print("TEST: \n", y1_test.value_counts(), end="\n\n")


TRAIN: 
 2. derogation                               1590
3. animosity                                1165
4. prejudiced discussions                    333
1. threats, plans to harm and incitement     310
Name: label_category, dtype: int64

VALIDATION: 
 2. derogation                               227
3. animosity                                167
4. prejudiced discussions                    48
1. threats, plans to harm and incitement     44
Name: label_category, dtype: int64

TEST: 
 2. derogation                               454
3. animosity                                333
4. prejudiced discussions                    94
1. threats, plans to harm and incitement     89
Name: label_category, dtype: int64



In [73]:
#vector that uses TreebankWordTokenizer without lemmatization
vector_no_lemma = CountVectorizer(tokenizer = TreebankWordTokenizer().tokenize, ngram_range=(1,2))

In [74]:
classifier = svm.LinearSVC(max_iter = 10000, class_weight= 'balanced')

# Create the pipeline

pipe_category = Pipeline([("cleaner", predictors()),
('vectorizer', vector_no_lemma),
('classifier', classifier)])

pipe_category.fit(x1_train, y1_train)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001BCAE2DBD90>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000001BCAE4B40D0>>)),
                ('classifier',
                 LinearSVC(class_weight='balanced', max_iter=10000))])

In [75]:
#evaluation metrics for the validation data

evaluation_metrics(x1_val, y1_val, pipe_category)

Accuracy: 0.5453
Precision: 0.5258
Recall: 0.4491
F1 Score: 0.4675


In [76]:
params = {'vectorizer__ngram_range': [(1,1),(1,2)],
          'classifier__C': [0.2, 0.4, 1],
          'classifier__class_weight': ['balanced', None]}

pipe1_optimized = GridSearchCV(pipe_category,param_grid=params,cv=10,scoring=make_scorer(f1_score, average="macro"),n_jobs=n_cpu-1,refit=True)
pipe1_optimized.fit(x1_train, y1_train)
print("Migliori iperparametri:",pipe1_optimized.best_params_)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Migliori iperparametri: {'classifier__C': 0.2, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2)}


In [77]:
best_params = pipe1_optimized.best_params_
pipe_category.set_params(**best_params)
pipe_category.fit(x1_train, y1_train)

d:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001BCAE2DBD90>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 2),
                                 tokenizer=<bound method TreebankWordTokenizer.tokenize of <nltk.tokenize.treebank.TreebankWordTokenizer object at 0x000001BCAE4B40D0>>)),
                ('classifier',
                 LinearSVC(C=0.2, class_weight='balanced', max_iter=10000))])

In [78]:
#evaluation metrics for the test data

evaluation_metrics(x1_test, y1_test, pipe_category)

Accuracy: 0.5124
Precision: 0.4932
Recall: 0.4438
F1 Score: 0.4597


In [26]:

import tkinter as tk
from PIL import ImageTk
from tkinter import *
import PIL.Image

if __name__ == '__main__':

    def getPrediction():
    
        input_text = text_box.get("1.0", "end-1c")
        input_text=[input_text]
        
        prediction = pipe_sexism.predict(input_text)
        pred_label.config(text="")
        
        if (prediction == 'sexist'):
            prediction = pipe_category.predict(input_text)

        pred_label.config(text=prediction[0])
        text_box.delete("1.0", tk.END)
        pred_label.grid(columnspan=3,column=1,row=10)


    root = tk.Tk()
    root.title("Sexism relevation")

    canv= tk.Canvas(root,width=600,height=300)
    canv.grid(columnspan=5,rowspan=20)

    logo = open("logo.png","rb")
    logo = PIL.Image.open(logo)
    logo = logo.resize((100,100))
    logo =ImageTk.PhotoImage(logo)
    logo_label = tk.Label(image = logo)
    logo_label.image = logo
    logo_label.grid(column=2,row=1,columnspan=1)


    istructions=tk.Label(root, text="Insert a message", font="Raleway")
    istructions.grid(columnspan=3,column=1,row=5)
    text_box = Text(root,height=3, width=40)
    text_box.config(state='normal')  
    text_box.grid(columnspan=3,column=1,row=8)

    pred_label=tk.Label(root, text="", font="Raleway")


    send_button = tk.Button(root, text ="Send", command = getPrediction)
    send_button.grid(columnspan=1,column=4,row=8)
    root.mainloop()


